Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 20344, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 20344 (delta 36), reused 12 (delta 12), pack-reused 20284 (from 4)
Receiving objects: 100% (20344/20344), 62.65 MiB | 7.92 MiB/s, done.
Resolving deltas: 100% (13676/13676), done.
/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 316.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements

In [2]:
#@title  Añadir soporte GGUF a ComfyUI
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/city96/ComfyUI-GGUF.git
%cd $WORKSPACE/custom_nodes/ComfyUI-GGUF
!pip install -r requirements.txt



/content/ComfyUI/custom_nodes
Cloning into 'ComfyUI-GGUF'...
remote: Enumerating objects: 612, done.
remote: Counting objects: 100% (324/324), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 612 (delta 270), reused 200 (delta 169), pack-reused 288 (from 1)
Receiving objects: 100% (612/612), 171.93 KiB | 946.00 KiB/s, done.
Resolving deltas: 100% (390/390), done.
/content/ComfyUI/custom_nodes/ComfyUI-GGUF
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 3.9 MB/s eta 0:00:00


In [3]:
#@title descargamos un modelo guff
%cd $WORKSPACE/models/unet
!wget -nc "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q3_K_S.gguf"

/content/ComfyUI/models/unet
--2025-05-14 17:39:56--  https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q3_K_S.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3e/5c/3e5c1f82226fc1460150126b5513fa266f4b8aea7194d1a31c944ad9da31ae10/f65bb921fe637ba9588278f54981afc567be408e38ee038d2fe3acf3e11188bd?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27flux1-dev-Q3_K_S.gguf%3B+filename%3D%22flux1-dev-Q3_K_S.gguf%22%3B&Expires=1747247997&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzI0Nzk5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNlLzVjLzNlNWMxZjgyMjI2ZmMxNDYwMTUwMTI2YjU1MTNmYTI2NmY0YjhhZWE3MTk0ZDFhMzFjOTQ0YWQ5ZGEzMWFlMTAvZjY1YmI5MjFmZTYzN2JhOTU4ODI3OGY1NDk4MWFmYzU2N2JlNDA4ZTM4

In [8]:
#@title Descarga de modelos (CLIP) requeridos para DualCLIPLoader (GGUF) con nombres correctos
%cd $WORKSPACE

!mkdir -p models/clip
%cd models/clip

# Descargar CLIP y renombrar a clip-vit-large-patch14.bin
!wget -nc -O CLIP-GmP-ViT-L-14.safetensors "https://huggingface.co/zer0int/CLIP-GmP-ViT-L-14/resolve/main/model.safetensors?download=true"

# Descargar T5 y renombrar a model.safetensors
!wget -nc -O t5-v1_1-xxl-encoder-Q4_K_S.gguf "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf?download=true"

# Verifica los archivos
!ls -lh $WORKSPACE/models/clip/


/content/ComfyUI
/content/ComfyUI/models/clip
File ‘CLIP-GmP-ViT-L-14.safetensors’ already there; not retrieving.
--2025-05-14 18:13:27--  https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf?download=true
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.55, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/48/8e/488e282fd263e7a5a8e462b6bf3e13fad39da3323736a1fe08cb84fb1f7f38aa/88b696cfae098f03bb078cc5944ef03aec1e91ec020a6b016b723a0f0532558c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27t5-v1_1-xxl-encoder-Q4_K_S.gguf%3B+filename%3D%22t5-v1_1-xxl-encoder-Q4_K_S.gguf%22%3B&Expires=1747250007&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzI1MDAwN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQ4Lzhl

In [5]:
#@title descargamos el vae
%cd $WORKSPACE/models/vae
!wget https://huggingface.co/StableDiffusionVN/Flux/resolve/main/Vae/flux_vae.safetensors

/content/ComfyUI/models/vae
--2025-05-14 17:42:44--  https://huggingface.co/StableDiffusionVN/Flux/resolve/main/Vae/flux_vae.safetensors
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/b8/a5/b8a57ba4d3c6088d6f4a9b539cdb398c3b22dd6fb56a7b4df77bf2f4911f4442/afc8e28272cd15db3919bacdb6918ce9c1ed22e96cb12c4d5ed0fba823529e38?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27flux_vae.safetensors%3B+filename%3D%22flux_vae.safetensors%22%3B&Expires=1747248165&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzI0ODE2NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2I4L2E1L2I4YTU3YmE0ZDNjNjA4OGQ2ZjRhOWI1MzljZGIzOThjM2IyMmRkNmZiNTZhN2I0ZGY3N2JmMmY0OTExZjQ0NDIvYWZjOGUyODI3MmNkMTVkYjM5MTliYWNkYjY5MThjZTljMWVkMjJlOTZ

In [9]:
#@title opcional pero recomendad Easy Use
#1. Clone the repo
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/yolain/ComfyUI-Easy-Use
#2. Install the requirements
%cd $WORKSPACE/custom_nodes/ComfyUI-Easy-Use
!pip install -r requirements.txt


/content/ComfyUI/custom_nodes
Cloning into 'ComfyUI-Easy-Use'...
remote: Enumerating objects: 4748, done.
remote: Counting objects: 100% (1235/1235), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 4748 (delta 1024), reused 950 (delta 946), pack-reused 3513 (from 2)
Receiving objects: 100% (4748/4748), 13.65 MiB | 20.71 MiB/s, done.
Resolving deltas: 100% (3248/3248), done.
/content/ComfyUI/custom_nodes/ComfyUI-Easy-Use
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.8/787.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00


In [ ]:
#@title Run ComfyUI with cloudflared (Recommended Way)
%cd $WORKSPACE
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

/content/ComfyUI
--2025-05-14 18:15:22--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.4.2/cloudflared-linux-amd64.deb [following]
--2025-05-14 18:15:23--  https://github.com/cloudflare/cloudflared/releases/download/2025.4.2/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/f6f8a9db-3887-4a1c-9ecb-9781f97395a5?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250514%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250514T181425Z&X-Amz-Expires=300&X-Amz-Signature=f56d913afc6aeee2f58a4244d816db0937c264734b2